In [121]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [122]:
decisions = pd.read_csv("../data/processed/decisions.csv")
books =  pd.read_csv("../data/processed/books.csv", index_col = "Unnamed: 0")
evaluation =  pd.read_csv("../data/external/evaluation.csv")

In [123]:
decisions_melt = pd.melt(decisions, id_vars = ["book_id", "decision"], value_vars=['recommendation_shown_0',
                                                                                   'recommendation_shown_1',
                                                                                   'recommendation_shown_2'])

In [124]:
decisions_melt.replace({"variable":{"recommendation_shown_0": "recommendation_0",
                                  "recommendation_shown_1": "recommendation_1",
                                  "recommendation_shown_2": "recommendation_2"}}, inplace = True)

In [125]:
decisions_melt["chosen"] = (decisions_melt["decision"] == decisions_melt["variable"]).astype(int)

In [127]:
decisions_grouped = decisions_melt.groupby(["book_id", "value"]).agg([('average','mean'),('chosen','sum')])
decisions_grouped.reset_index(inplace= True)

In [128]:
decisions_grouped.columns = ['book_id', 'recommendation_id', 'average', 'chosen']

In [131]:
recommendation = {}
for book_id in evaluation.itemID:
    recommendation[book_id] = []

In [132]:
def update_recommendation(level, recommendation):
    for i in tqdm(range(level.shape[0])):
            book_id = level.book_id[i]
            recommendation_id = level.recommendation_id[i]
            if recommendation_id not in recommendation[book_id] and len(recommendation[book_id]) < 5:
                recommendation[book_id].append(recommendation_id)

In [133]:
level_one = decisions_grouped[
    np.logical_and(decisions_grouped.chosen > 1,
                   decisions_grouped.average > 0.5)].sort_values(["average","chosen"],
                                                                 ascending = False).reset_index(drop = True)

In [134]:
update_recommendation(level_one, recommendation)

100%|██████████| 1659/1659 [00:00<00:00, 36142.96it/s]


In [135]:
level_two = decisions_grouped[
    np.logical_or(decisions_grouped.chosen == 1,
                  decisions_grouped.average <= 0.5)].sort_values(["average","chosen"],
                                                                 ascending = False).reset_index(drop = True)

In [136]:
update_recommendation(level_two, recommendation)

100%|██████████| 17086/17086 [00:00<00:00, 78547.98it/s]


In [137]:
result = pd.DataFrame(recommendation).transpose().reset_index().rename({"index":"book_id",
                                                               0:"recommendation_1",
                                                               1:"recommendation_2",
                                                               2:"recommendation_3",
                                                               3:"recommendation_4",
                                                               4:"recommendation_5"}, axis = 1)

In [138]:
result["team_id"] = "dataminerz"
result["model_id"] = "ensemble"

In [139]:
result.to_csv("../data/processed/ensemble_dataminerz.csv")